# Installing Packages

In [ ]:
# 주의사항: 항상 GPU T4 x2를 사용해야 함
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

In [ ]:
# 외부 패키지 import
%pip install /kaggle/input/lmsys-packages/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
%pip install /kaggle/input/lmsys-packages/xformers-0.0.24042abc8.d20240802-cp310-cp310-linux_x86_64.whl
%pip install /kaggle/input/some-pack/faiss_cpu_downloads/faiss_cpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
%pip install --no-index --find-links=/kaggle/input/some-pack/sentence_transformers_packages sentence-transformers

In [ ]:
!cp -r /kaggle/input/lmsys-modules-0805 human_pref

## Check input files

In [ ]:
import os

# /kaggle/input 폴더 아래 모든 디렉터리/파일 순회
for dirname, _, filenames in os.walk('/kaggle/input'):
    # 현재 디렉토리의 각 파일 순회
    for filename in filenames:
        # 디렉터리와 파일명 연결해 파일의 전체 경로 출력
        print(os.path.join(dirname, filename))

# Prepare test file

In [ ]:
%%writefile test.py
import pandas as pd

# Kaggle 입력 디렉터리에서 원본 테스트 CSV 파일 로드
df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# 모델 A가 항상 승자임을 나타내는 더미 레이블 열 추가
df["winner_model_a"] = 1
df["winner_model_b"] = 0
df["winner_tie"] = 0

## TODO: 어째서 이렇게 하는가?
# 원본 테스트 데이터프레임(가짜 레이블 포함)을 parquet 파일로 저장
df.to_parquet("test.parquet", index=False)

# 응답 A와 B 교환(역방향 비교 시뮬레이션)
df["response_a"], df["response_b"] = df["response_b"], df["response_a"]

# 스왑된 버전을 별도의 parquet 파일로 저장
df.to_parquet("test_swap.parquet", index=False)

In [ ]:
!python test.py

# Inference: gemma2-9b

In [ ]:
%%writefile gemma.py
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer

from xformers.ops.fmha.attn_bias import BlockDiagonalCausalMask
from human_pref.models.modeling_gemma2 import Gemma2ForSequenceClassification
from human_pref.data.processors import ProcessorPAB
from human_pref.data.dataset import LMSYSDataset
from human_pref.data.collators import VarlenCollator, ShardedMaxTokensCollator
from human_pref.utils import to_device

# 사전 훈련된 모델 위치+입력 데이터 경로
model_name_or_path = "/kaggle/input/lmsys-checkpoints-0-0805"  
csv_path = "test.parquet"  

# 토크나이저와 프로세서 로드
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# 입력 샘플(프롬프트-응답 쌍) 토큰화하는 프로세서
processor = ProcessorPAB(
    tokenizer=tokenizer,
    max_length=5120,
    support_system_role=False,
)

# 데이터셋 로드하고 전처리 적용
dataset = LMSYSDataset(
    csv_file=csv_path,
    query=None,
    processor=processor,
    include_swap=False,
    is_parquet=True,
)

# 토큰 총 개수에 따라 예제를 배치 처리하는 사용자 정의 정렬기를 사용한 DataLoader
# 각 배치는 마이크로 배치들의 목록
# max_tokens: 배치당 최대 토큰 총량
dataloader = DataLoader(
    dataset,
    batch_size=80,  
    num_workers=4,
    collate_fn=ShardedMaxTokensCollator(
        max_tokens=8192,
        base_collator=VarlenCollator()
    ),
)

# GPU 2개 파이프라인 병렬 처리 정의
# GPU 2개의 멀티프로세싱 성능을 극대화하기 위한 작업

# 모델 내 트랜스포머 레이어의 수
num_hidden_layers = 42

# 임베딩/최종 레이어/스코어 헤드를 GPU 0과 1에 할당
device_map = {
    "model.embed_tokens": "cuda:0",
    "model.norm": "cuda:1",
    "score": "cuda:1",
}

# 모델 레이어를 두 개의 GPU로 분할(첫 절반은 cuda:0, 두 번째 절반은 cuda:1)
for i in range(num_hidden_layers // 2):
    device_map[f"model.layers.{i}"] = "cuda:0"
for i in range(num_hidden_layers // 2, num_hidden_layers):
    device_map[f"model.layers.{i}"] = "cuda:1"

# 가중치가 포함된 모델을 장치에 로드하고 float16을 사용
model = Gemma2ForSequenceClassification.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# 회전 임베딩 준비

config = model.config
# 어텐션 헤드의 차원
dim = config.head_dim
# RoPE(회전 위치 인코딩)에 대한 역주파수 계산
inv_freq = 1.0 / (
    config.rope_theta ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim)
)
# 각각 모델 초반, 후반
inv_freq0 = inv_freq.to("cuda:0")
inv_freq1 = inv_freq.to("cuda:1")

# 파이프라인 실행을 이용한 추론 루프
is_first = True
hidden_states = None
outs = []

# 전체 배치에 대해서
for batch in tqdm(dataloader):
    # 각 배치는 마이크로배치의 리스트
    for micro_batch in batch:
        # 입력 토큰을 GPU 0으로 이동
        input_ids = to_device(micro_batch["input_ids"], "cuda:0")

        # 시퀀스 관련 정보 준비
        seq_info = dict(
            cu_seqlens=micro_batch["cu_seqlens"],
            position_ids=micro_batch["position_ids"],
            max_seq_len=micro_batch["max_seq_len"],
            attn_bias=BlockDiagonalCausalMask.from_seqlens(micro_batch["seq_lens"]),
        )
        seq_info = to_device(seq_info, "cuda:0")

        # 첫 실행에서는 파트 1만 실행하고 상태 저장
        if is_first:
            with torch.no_grad(), torch.cuda.amp.autocast():
                prev_hidden_states = model.forward_part1(input_ids, seq_info, inv_freq1)
            is_first = False
            # 다음 단계 위해 중간 출력을 GPU 1로 이동
            prev_seq_info, prev_hidden_states = to_device(
                [seq_info, prev_hidden_states], "cuda:1"
            )
            continue

        # 이전 마이크로배치에 대해 2부 실행하고 현재에 대해 1부 실행
        with torch.no_grad(), torch.cuda.amp.autocast():
            # 이전 숨겨진 상태에 대한 최종 logit값 계산
            # logit: 어떤 사건의 성공 확률을 나타내는 P(A)와 실패 확률을 나타내는 1-P(A)의 비율의 로그값
            logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)

            # 다음 마이크로배치에 대한 숨겨진 상태를 계산
            hidden_states = model.forward_part1(input_ids, seq_info, inv_freq1)

            # 새로운 숨겨진 상태와 seq_info를 GPU 1로 이동
            prev_seq_info, prev_hidden_states = to_device(
                [seq_info, hidden_states], "cuda:1"
            )
            # 예측 logit 값을 CPU에 저장
            outs.append(logits.cpu())

# 마지막 마이크로배치에 대한 최종 예측
with torch.no_grad(), torch.cuda.amp.autocast():
    logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
    outs.append(logits.cpu())

# 모든 logit을 연결하고 확률 계산
pred = torch.cat(outs, dim=0)
prob = pred.softmax(-1)

# 데이터셋의 내장 함수로 예측을 평가
print(dataset.evaluate(prob.numpy()))

# 예측 확률을 파일에 저장
np.save('prob_m0.npy', prob)


In [ ]:
!python gemma.py

# Inference: llama3-8b

In [ ]:
%%writefile llama.py
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer

from xformers.ops.fmha.attn_bias import BlockDiagonalCausalMask
from human_pref.models.modeling_llama import LlamaForSequenceClassification
from human_pref.data.processors import ProcessorPAB
from human_pref.data.dataset import LMSYSDataset
from human_pref.data.collators import VarlenCollator, ShardedMaxTokensCollator
from human_pref.utils import to_device

# 사전 훈련된 모델 위치+입력 데이터 경로
model_name_or_path = "/kaggle/input/lmsys-checkpoints-3-0805"
csv_path = "test_swap.parquet"

# 토크나이저와 프로세서 로드
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# 토크나이저 경고 무시
tokenizer.deprecation_warnings[
    "sequence-length-is-longer-than-the-specified-maximum"
] = True

# 입력 샘플(프롬프트-응답 쌍) 토큰화하는 프로세서
processor = ProcessorPAB(
    tokenizer=tokenizer,
    max_length=5120,
    support_system_role=True,
)

# 데이터셋 로드하고 전처리 적용
dataset = LMSYSDataset(
    csv_file=csv_path,
    query=None,
    processor=processor,
    include_swap=False,
    is_parquet=True,
)

# 토큰 총 개수에 따라 예제를 배치 처리하는 사용자 정의 정렬기 사용한 DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=80,
    num_workers=4,
    collate_fn=ShardedMaxTokensCollator(
        max_tokens=8192,
        base_collator=VarlenCollator()
    ),
)

# 파이프라인 병렬 처리를 위한 장치 매핑: LLaMA-3는 32개의 트랜스포머 레이어가 존재
num_hidden_layers = 32

device_map = {
    "model.embed_tokens": "cuda:0",
    "model.norm": "cuda:1",
    "score": "cuda:1",
}

# 모델 레이어를 두 개의 GPU로 분할(첫 절반은 cuda:0, 두 번째 절반은 cuda:1)
for i in range(num_hidden_layers // 2):
    device_map[f"model.layers.{i}"] = "cuda:0"
for i in range(num_hidden_layers // 2, num_hidden_layers):
    device_map[f"model.layers.{i}"] = "cuda:1"

# 가중치가 포함된 모델을 장치에 로드하고 float16을 사용
model = LlamaForSequenceClassification.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# 회전 임베딩 준비
config = model.config
dim = config.hidden_size // config.num_attention_heads

# RoPE(회전 위치 인코딩)에 대한 역주파수 계산
inv_freq = 1.0 / (
    config.rope_theta ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim)
)
inv_freq0 = inv_freq.to("cuda:0")
inv_freq1 = inv_freq.to("cuda:1")

# 파이프라인 실행을 이용한 추론 루프
is_first = True
hidden_states = None
outs = []

# 전체 배치에 대해서
for batch in tqdm(dataloader):
    for micro_batch in batch:
        # 입력 토큰을 GPU 0으로 이동
        input_ids = to_device(micro_batch["input_ids"], "cuda:0")

        # 시퀀스 관련 정보 준비
        seq_info = dict(
            cu_seqlens=micro_batch["cu_seqlens"],
            position_ids=micro_batch["position_ids"],
            max_seq_len=micro_batch["max_seq_len"],
            attn_bias=BlockDiagonalCausalMask.from_seqlens(micro_batch["seq_lens"]),
        )
        seq_info = to_device(seq_info, "cuda:0")

        # 첫 실행에서는 파트 1만 실행하고 상태 저장
        if is_first:
            with torch.no_grad(), torch.cuda.amp.autocast():
                prev_hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            is_first = False
            # 다음 단계 위해 중간 출력을 GPU 1로 이동
            prev_seq_info, prev_hidden_states = to_device(
                [seq_info, prev_hidden_states], "cuda:1"
            )
            continue

        # 이전 마이크로배치에 대해 2부 실행하고 현재에 대해 1부 실행
        with torch.no_grad(), torch.cuda.amp.autocast():
            # 이전 숨겨진 상태에 대한 최종 logit값 계산
            logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
            hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            # 다음 마이크로배치에 대한 숨겨진 상태를 계산
            prev_seq_info, prev_hidden_states = to_device(
                [seq_info, hidden_states], "cuda:1"
            )
            outs.append(logits.cpu())

# --- Process final micro-batch (no part1 needed) ---
with torch.no_grad(), torch.cuda.amp.autocast():
    logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
    outs.append(logits.cpu())

# 모든 logit을 연결하고 확률 계산
pred = torch.cat(outs, dim=0)        
prob = pred.softmax(-1)              
print(dataset.evaluate(prob.numpy()))

# 예측 확률을 파일에 저장
np.save('prob_m3.npy', prob)
print("prob_m3 saved")

In [ ]:
!python llama.py

In [ ]:
import numpy as np

prob = np.load('prob_m3.npy')

print(prob[:5])


# Inference: Faiss

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import AutoTokenizer
from sklearn.preprocessing import MinMaxScaler
import faiss

model_load_path = '/kaggle/input/some-pack/sentence-transformer-model' 
sentence_model = SentenceTransformer(model_load_path)

test_data = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

class CustomDebertaModel(nn.Module):
    def __init__(self, model_name, num_labels, feature_dim=2, dropout_rate=0.1):
        super(CustomDebertaModel, self).__init__()
        
        # DeBERTa 모델 초기화
        self.base_model = AutoModel.from_pretrained(model_name)
        
        # 유사성 특징을 위한 특징 타워 (소규모 MLP)
        self.feature_fc = nn.Sequential(
            # 입력 유사성 특징을 128차원 공간에 매핑
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            # 텍스트 임베딩과 동일한 크기로 프로젝트 조정
            nn.Linear(128, self.base_model.config.hidden_size),  
            nn.ReLU()
        )
        
        # 텍스트와 유사성 임베딩 간의 상호작용을 허용하는 주의 메커니즘
        self.attention = nn.MultiheadAttention(
            embed_dim=self.base_model.config.hidden_size,
            # 어텐션 헤드 수
            num_heads=4,  
            # 배치 우선 입력 포맷 활성화
            batch_first=True  
        )
        
        # 정규화를 위한 드롭아웃 레이어
        self.dropout = nn.Dropout(p=dropout_rate)

        # 최종 분류기 레이어 (분류를 위한 MLP)
        self.classifier = nn.Sequential(
            nn.Linear(self.base_model.config.hidden_size * 2, self.base_model.config.hidden_size),  # Combine text + attention features
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(self.base_model.config.hidden_size, num_labels)
        )

    def forward(self, input_ids, attention_mask, similarity_features, labels=None):
        # 텍스트 타워: DeBERTa에서 [CLS] 토큰 임베딩 추출
        base_outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        text_embeddings = base_outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

        # 특징 타워: MLP를 통한 프로세스 유사성 특징 처리
        similarity_embeds = self.feature_fc(similarity_features)  # [batch_size, hidden_size]

        # 주의 메커니즘을 활용한 교차 모달 상호작용
        query = text_embeddings.unsqueeze(1)       # 형상: [batch_size, 1, hidden_size]
        key_value = similarity_embeds.unsqueeze(1) # 형상: [batch_size, 1, hidden_size]
        attention_output, _ = self.attention(query, key_value, key_value)  # [batch_size, 1, hidden_size]

        # 텍스트와 어텐디드 유사성 특징을 결합
        combined_features = torch.cat([text_embeddings, attention_output.squeeze(1)], dim=1)

        # 드롭아웃 및 분류 헤드를 적용
        logits = self.classifier(self.dropout(combined_features))

        # logit 값을 포함한 출력 사전
        outputs = {"logits": logits}
        
        # 라벨이 제공되는 경우, 교차 엔트로피 손실을 계산
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            outputs["loss"] = loss_fn(logits, labels)

        return outputs

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import faiss

test_data = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

# FAISS 사용해 의미적 유사도 점수 계산
def compute_semantic_features_with_faiss(df):
    # 프롬프트와 응답을 목록으로 추출
    prompts = df['prompt'].tolist()
    responses_a = df['response_a'].tolist()
    responses_b = df['response_b'].tolist()

    # 문장 임베딩을 생성하고 정규화(단위 벡터)
    prompt_embeddings = np.array(sentence_model.encode(prompts))
    prompt_embeddings = prompt_embeddings / np.linalg.norm(prompt_embeddings, axis=1, keepdims=True)

    response_a_embeddings = np.array(sentence_model.encode(responses_a))
    response_a_embeddings = response_a_embeddings / np.linalg.norm(response_a_embeddings, axis=1, keepdims=True)

    response_b_embeddings = np.array(sentence_model.encode(responses_b))
    response_b_embeddings = response_b_embeddings / np.linalg.norm(response_b_embeddings, axis=1, keepdims=True)

    # 임베딩 차원 결정
    dim = prompt_embeddings.shape[1]
    
    # 내적(코사인 유사도) 사용하여 FAISS 인덱스 생성
    index_flat = faiss.IndexFlatIP(dim)

    # 응답 A와 프롬프트 간의 유사도 계산
    # FAISS 인덱스에 프롬프트 임베딩 추가
    # 상위 1개 유사도 점수 획득
    index_flat.add(prompt_embeddings) 
    similarity_a = index_flat.search(response_a_embeddings, k=1)[0].squeeze()  

    # B에 대한 유사도 재설정 및 계산
    index_flat.reset()
    index_flat.add(prompt_embeddings)
    similarity_b = index_flat.search(response_b_embeddings, k=1)[0].squeeze()

    # 데이터프레임에 유사도 점수 저장
    df['similarity_a'] = similarity_a
    df['similarity_b'] = similarity_b

    return df

In [ ]:
test_data = compute_semantic_features_with_faiss(test_data)

In [ ]:
# 훈련된 사용자 정의 PyTorch 모델을 로드
model = torch.load("/kaggle/input/akemiiiiii/custom_model_dir/custom_model_complete.pth")

# 사용 가능한 경우 장치를 GPU로 설정하고 그렇지 않으면 CPU로 대체
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# 중요: 평가 모드 설정
model.eval()  

print("Custom model loaded successfully!")

# 훈련 중 사용된 토크나이저 로드
tokenizer_path = "/kaggle/input/akemiiiiii/custom_model_dir"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# 단일 테스트 샘플 전처리 함수
def preprocess_test_data(row):
    # 훈련과 동일한 형식으로 입력 문자열 구성
    input_text = f"Prompt: {row['prompt']} Response A: {row['response_a']} Response B: {row['response_b']}"

    # 훈련과 동일한 토크나이저 설정으로 입력 토큰화
    tokenized_inputs = tokenizer(
        input_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"  # PyTorch 텐서 반환
    )

    # 유사도 점수를 추가 특징으로 추가
    # 올바른 배치 처리를 위해 [1, feature_dim] 형식으로 구성해야 함
    similarity = torch.tensor([[row["similarity_a"], row["similarity_b"]]], dtype=torch.float32)

    tokenized_inputs["similarity_features"] = similarity

    return tokenized_inputs

# 테스트 데이터셋의 모든 행에 전처리 함수 적용
processed_test_data = [preprocess_test_data(row) for _, row in test_data.iterrows()]

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

test_dataset = TestDataset(processed_test_data)

def collate_fn_test(batch):
    # 입력 ID와 어텐션 마스크를 배치 차원을 따라 결합
    input_ids = torch.cat([item["input_ids"] for item in batch], dim=0)
    attention_mask = torch.cat([item["attention_mask"] for item in batch], dim=0)
    
    # 스택 유사성 특징
    similarity_features = torch.cat([item["similarity_features"] for item in batch], dim=0)  # shape [B, 2]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "similarity_features": similarity_features,
    }

test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn_test, shuffle=False)

In [ ]:
# 모델을 평가모드로 설정(드롭아웃 비활성화 필수)
model.eval()

# 예측 결과 저장 리스트
predictions = []

# 속도 빠르게 하기 위한 경사 계산 비활성화
with torch.no_grad():
    for batch in test_dataloader:
        # 배치 데이터를 올바른 장치(GPU/CPU)로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        similarity_features = batch["similarity_features"].to(device)

        # 전방 통과 실행(모델 추론)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            similarity_features=similarity_features
        )

        # 출력에서 원시 logit 값 추출
        logits = outputs["logits"]

        # 소프트맥스로 logit을 확률로 변환
        probs = torch.nn.functional.softmax(logits, dim=-1)

        # CPU로 이동하고 나중에 사용하기 위해 NumPy로 변환
        predictions.append(probs.cpu().numpy())

# 모든 배치 결과를 하나의 NumPy 배열로 결합클릭하여 적용
predictions = np.concatenate(predictions, axis=0)

# 최종 예측 출력
print(predictions)

# 예측을 .npy에 저장
np.save('prob_faiss.npy', predictions)

# Create submission file

In [ ]:
df = pd.read_parquet("test.parquet")


prob_m0 = np.load("prob_m0.npy")  # Gemma2
prob_m3 = np.load("prob_m3.npy")[:, [1, 0, 2]]  # Llama3
prob_faiss = np.load("prob_faiss.npy")  # faiss

# 예측값과 가중치를 결합
# 최적의 성능을 위해 가중치 조정
preds = np.average(
    [
        prob_m0,       # Gemma2 결과
        prob_m3,       # Llama3 결과
        prob_faiss     # faiss 결과
    ],
    axis=0,
    weights=[0.5, 0.5, 0.0]  # 각 모델 가중치
)

# 제출용
sub = pd.DataFrame({
    "id": df["id"],
    "winner_model_a": preds[:, 0],
    "winner_model_b": preds[:, 1],
    "winner_tie": preds[:, 2],
})

# Save to CSV
sub.to_csv("submission.csv", index=False)
print(sub.head())


# Reference
- LMSYS - Chatbot Arena Human Preference Predictions 2nd place solution - https://www.kaggle.com/competitions/lmsys-chatbot-arena/discussion/527685
- Blue - https://www.kaggle.com/code/blue0924/finetuning-test2